In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
challenge_cities_df = pd.read_csv("WeatherPy_vacation.csv")
challenge_cities_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,6,Mahibadhoo,MV,2020-02-20 04:25:11,3.78,72.97,81.66,74,0,17.83,clear sky,0.0,0.0
1,11,Rikitea,PF,2020-02-20 04:25:12,-23.12,-134.97,78.62,79,21,13.11,few clouds,0.0,0.0
2,16,Busselton,AU,2020-02-20 04:25:13,-33.65,115.33,91.99,17,0,12.97,clear sky,0.0,0.0
3,19,Padang,ID,2020-02-20 04:25:14,-0.95,100.35,79.05,61,43,7.72,scattered clouds,0.0,0.0
4,26,Kayes,ML,2020-02-20 04:25:16,14.00,-11.00,75.20,33,0,2.24,clear sky,0.0,0.0


In [3]:
# Get the data types.
challenge_cities_df.dtypes

Unnamed: 0                    int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = challenge_cities_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = challenge_cities_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = challenge_cities_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = challenge_cities_df[["Lat", "Lng"]]
humidity = challenge_cities_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = challenge_cities_df[["Lat", "Lng"]]
clouds = challenge_cities_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent wind speed
locations = challenge_cities_df[["Lat", "Lng"]]
wind = challenge_cities_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = challenge_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahibadhoo,MV,81.66,clear sky,3.78,72.97,
1,Rikitea,PF,78.62,few clouds,-23.12,-134.97,
2,Busselton,AU,91.99,clear sky,-33.65,115.33,
3,Padang,ID,79.05,scattered clouds,-0.95,100.35,
4,Kayes,ML,75.20,clear sky,14.00,-11.00,
...,...,...,...,...,...,...,...
112,Ahipara,NZ,82.99,overcast clouds,-35.17,173.17,
113,Port-Gentil,GA,80.29,overcast clouds,-0.72,8.78,
114,Benjamin Constant,BR,78.80,scattered clouds,-4.38,-70.03,
115,Epe,NG,76.69,scattered clouds,6.58,3.98,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df = hotel_df
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahibadhoo,MV,81.66,clear sky,3.78,72.97,Omadhoo Inn
1,Rikitea,PF,78.62,few clouds,-23.12,-134.97,Pension Maro'i
2,Busselton,AU,91.99,clear sky,-33.65,115.33,Observatory Guest House
3,Padang,ID,79.05,scattered clouds,-0.95,100.35,Mercure Padang
4,Kayes,ML,75.20,clear sky,14.00,-11.00,
...,...,...,...,...,...,...,...
112,Ahipara,NZ,82.99,overcast clouds,-35.17,173.17,Ahipara Holiday Park
113,Port-Gentil,GA,80.29,overcast clouds,-0.72,8.78,Fengshui Residence
114,Benjamin Constant,BR,78.80,scattered clouds,-4.38,-70.03,CASA AVALIADA
115,Epe,NG,76.69,scattered clouds,6.58,3.98,MAGVILLE HOTEL


In [14]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Add a marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

In [17]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))